In [1]:
import sys
sys.path.append("fwks_lib")

# limit the memory usage - this actually uses very small amount of memory
import keras.backend as K
import tensorflow as tf
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.75)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
K.set_session(sess)

In [2]:
# generate denoising models

from fwks.tasks import Task, make_training_task
from fwks.model import DenoisingModel
from fwks import stage
from fwks.noise_gen import Static

def mk_model():
    return DenoisingModel([
        stage.Window(512, 256),
        stage.LogPowerFourier(),
        stage.RNN(width=257, depth=3),
        stage.L2Loss()
    ])


# this does not generate recordings for some reason

model = mk_model()
DenoisingTask = make_training_task(Static())
DenoisingTask.how_much = 80
DenoisingTask.mock = True


class MkDenoisingModel(metaclass=DenoisingTask):
    @classmethod
    def get_acoustic_model(self):
        return model


/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Yet another class to be finished: TrainableCZT
Yet another class to be finished: TuningCurves
Yet another class to be finished: RoEx
Yet another class to be finished: CARFAC
Yet another class to be finished: GammaChirp
Yet another class to be finished: TimeRoex
Yet another class to be finished: TrainableWavelet
Yet another class to be finished: CARFAC
Yet another class to be finished: AntimonotonyLayer
Yet another class to be finished: SpectralCoefficients
Yet another class to be finished: Pointwise
Yet another class to be finished: AbstractWavelet
Yet another class to be finished: AbstractFilter
Yet another class to be finished: ExcitationTrace
Yet another class to be finished: TrainableCZT
Yet another class to be finished: TuningCurves
Yet another class to be finished: RoEx
Yet another class to be finished: CARFAC
Yet another class to be finished: GammaChirp
Yet another class to be finished: TimeRoex
Yet another class to be finished: TrainableWavelet
Yet another class to be finished:

/venv/lib/python3.5/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
MkDenoisingModel.run("/tmp/newcache")

0it [00:00, ?it/s]

Getting dataset lengths


ValueError: max() arg is an empty sequence

In [ ]:
from fwks.tasks import AbstractModelTraining

denoising_model = MkDenoisingModel.get_model()


class BaselineMFCCModel(metaclass=AbstractModelTraining):
    @classmethod
    def get_acoustic_model(self):
        return mfcc_model(denoising_model)


class HandcraftedModel(metaclass=AbstractModelTraining):
    @classmethod
    def get_acoustic_model(self):
        return handcrafted_model(denoising_model)

    
class PLCModel(metaclass=AbstractModelTraining):
    @classmethod
    def get_acoustic_model(self):
        return plc_model(denoising_model)

    
class MelFBModel(metaclass=AbstractModelTraining):
    @classmethod
    def get_acoustic_model(self):
        return melfilterbank(denoising_model)

    
class CochleagramModel(metaclass=AbstractModelTraining):
    @classmethod
    def get_acoustic_model(self):
        return cochleagram(denoising_model)
